#### Import libraries¶

In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from datetime import datetime, timedelta
import matplotlib

#### Import data

In [2]:
master_data = pd.read_csv('master_springsteen_data.csv')
master_data.drop('Unnamed: 0', axis=1, inplace=True)
data = master_data

#### Create a dictionary of words and their frequency for each album

In [3]:
lyrics_list = map(lambda item: item.replace("'", "").replace('"','').replace(',,', ','), data.loc[:,'Lyrics'])

In [4]:
for song in range(0,len(lyrics_list)):
    lyrics_list[song] = lyrics_list[song].lower()

In [5]:
flatten = lambda l: [item for sublist in l for item in sublist]

lyrics_words = []
album_words = []
all_words = []

for i in range(0,len(data)):   
    if i > 0:
        if data.loc[i,'Album'] != data.loc[(i-1),'Album']:
            album_words.append(flatten(lyrics_words))
            lyrics_words = [lyrics_list[i].replace(',,' , ',').split(',')]
            all_words.append( lyrics_list[i].replace(',,' , ',').split(',') )
        else: 
            lyrics_words.append( lyrics_list[i].replace(',,' , ',').split(',') )
            all_words.append( lyrics_list[i].replace(',,' , ',').split(',') )
    else: 
        lyrics_words.append( lyrics_list[i].replace(',,' , ',').split(',') ) 
        all_words.append( lyrics_list[i].replace(',,' , ',').split(',') )

In [6]:
album_word_dict = []

for album in range(0,len(album_words)):
    word_string = str(album_words[album]).replace("'","").replace('\\','').replace('.','').\
    replace('[','').replace(']','').replace('(','').replace(')','').replace('?','').replace(' ','')
    word_string = word_string.split(',')
    album_word_dict.append(dict(Counter(word_string))) 

#### Create a list of lists, where each sublist contains the words in each album, ordered from most to least frequent

In [7]:
words_by_frequency = []
for album in range(0,len(album_word_dict)):
    sorted_list = list(reversed(sorted(album_word_dict[album], key=album_word_dict[album].get)))
    words_by_frequency.append(sorted_list)

#### Create a dictionary of every word in all songs, where the word is the key and its frequency is the value

In [8]:
flatten(all_words)
all_words_string = str(all_words).replace("'","").replace('\\','').replace('.','').\
replace('[','').replace(']','').replace('(','').replace(')','').replace('?','').replace(' ', '')
all_words_string = all_words_string.split(',')
all_words_dict = dict(Counter(all_words_string))

#### Create a list of lists, similar to 'words_by_frequency', but with words excluded if they appear more than an average of n times in all other albums. For example, if 'drive' is one of the most frequent words in 'Born to Run' but it appears, on average, n times or more in other albums, it would be removed from the list. This function can be tuned by adjusting the value of 'n'.

In [9]:
unique_words = [[] for i in range(len(words_by_frequency))]
uniqueness_values = [[] for i in range(len(words_by_frequency))]

for album in range(0,len(words_by_frequency)):
    for word in range(0,len(words_by_frequency[album])):
        uniqueness = (all_words_dict[words_by_frequency[album][word]] - album_word_dict[0][words_by_frequency[0][100]] ) / float(20)
        uniqueness_values[album].append(uniqueness)
        if uniqueness < 4:
            unique_words[album].append(words_by_frequency[album][word])
        else: pass      

#### Create a DataFrame where each row represents a unique album. The album names in this DataFrame will be used as the column names in the 'characteristic_words' DataFrame below.¶

In [10]:
albums = data[['Album','Song']].groupby('Album', as_index=False, sort=False).count()
albums.head()

,Album,Song
0,"Greetings from Asbury Park, N.J.",9
1,"The Wild, the Innocent, and the E Street Shuffle",7
2,Born to Run,8
3,Darkness on the Edge of Town,10
4,The River,20


#### Create a 'characteristic_words' DataFrame from the 'unique_words' list of lists

In [11]:
characteristic_words = pd.DataFrame({albums.loc[0,'Album']:unique_words[0]})

In [12]:
for album in range(1,len(album_word_dict)):
    characteristic_words = characteristic_words.merge(pd.DataFrame({albums.loc[album,'Album']:unique_words[album]}), \
                                                      left_index=True, right_index=True)

In [13]:
characteristic_words.head()

,"Greetings from Asbury Park, N.J.","The Wild, the Innocent, and the E Street Shuffle",Born to Run,Darkness on the Edge of Town,The River,Nebraska,Born in the U.S.A.,Tunnel of Love,Human Touch,Lucky Town,The Ghost of Tom Joad,18 Tracks,The Rising,Devils & Dust,We Shall Overcome: The Seeger Sessions,Magic,Working on a Dream,The Promise,Wrecking Ball,High Hopes
0,came,them,shes,prove,better,him,glory,alone,real,days,across,something,may,about,blowed,has,surprise,whoa,ground,raise
1,mary,johnny,backstreets,these,darlin,everything,days,honey,nothin,these,youngstown,dum,sky,thinkin,uh-huh,youll,youve,da,rocky,hopes
2,could,boys,run,working,ooh,stop,usa,look,lovin,better,border,usa,empty,nothin,lord,alive,seen,sha,drawn,high
3,too,wont,hey,badlands,youve,put,born,two,mans,lucky,line,born,li,bed,hold,comin,working,breakaway,shackled,harry_s
4,sure,train,freeze-out,till,point,sir,hometown,gone,job,beautiful,highway,thats,faith,god,pay,die,itself,darling,ball,would


#### Export 'characteristic_words' to a csv file. This file will be formatted and modified to include the top 10 rows in Excel.

In [14]:
characteristic_words.to_csv('characteristic_words.csv')